In [ ]:
!pip install webdriver-manager

In [1]:
import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time

from IPython.display import clear_output

import re

import math

In [ ]:
latin_stress_to_standard = {
    "á": "a",
    "é": "e",
    "í": "i",
    "ó": "o",
    "ú": "u",
}

### 20minutos

In [ ]:
PATH = "C:\Program Files (x86)\chromedriver.exe"
JOURNAL_URL = "https://www.20minutos.es/"

In [ ]:
DEFAULT_WAIT_TIME = 5
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=None)
driver.get(JOURNAL_URL)
clickable_condition = EC.element_to_be_clickable((By.XPATH, "//button/span[contains(text(), 'Aceptar ')]"))
WebDriverWait(driver, DEFAULT_WAIT_TIME).until(clickable_condition).click()

Extract hrefs

In [ ]:
areas_href = driver.find_elements(By.XPATH, "//ul[@class='default-menu']/li//a")
areas_name = [a.get_attribute("href").split("/")[-2] for a in areas_href]
areas_name

In [64]:
PATH = "C:\Program Files (x86)\chromedriver.exe"
JOURNAL_URL = "https://www.20minutos.es/"
JOURNAL_URL = "https://okdiario.com/"

In [ ]:
latin_stress_to_standard = {
    "á": "a",
    "é": "e",
    "í": "i",
    "ó": "o",
    "ú": "u",
}

In [66]:
DEFAULT_WAIT_TIME = 5
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=None)
driver.get(JOURNAL_URL)
clickable_condition = EC.element_to_be_clickable((By.XPATH, "//button/span[contains(text(), 'Aceptar ')]"))
WebDriverWait(driver, DEFAULT_WAIT_TIME).until(clickable_condition).click()

In [ ]:
areas_href = driver.find_elements(By.XPATH, "//ul[@class='default-menu']/li//a")
areas_name = [a.get_attribute("href").split("/")[-2] for a in areas_href]
areas_name

In [ ]:
all_page_urls = driver.find_elements(By.XPATH, "//a[@href]")

In [ ]:
all_page_urls

### OKdiario

In [ ]:
DEFAULT_WAIT_TIME = 5
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=None)
driver.get(JOURNAL_URL)
clickable_condition = EC.element_to_be_clickable((By.XPATH, "//button/span[contains(text(), 'Aceptar ')]"))
WebDriverWait(driver, DEFAULT_WAIT_TIME).until(clickable_condition).click()

Extract hrefs

In [175]:
def extract_articles_meta():
    all_articles = driver.find_elements(By.XPATH, "//article[@id]")
    all_page_ids = [elem.get_attribute("id") for elem in all_articles]
    all_page_urls = [elem.find_element(By.XPATH, "/a[@href]/header/h2").text for elem in all_articles]
    all_page_titles = [elem.find_element(By.XPATH, "/a/header[@class='article-header']") for elem in all_articles]
    all_page_author_date = [elem.find_element(By.XPATH, "/a/footer[@class='article-footer']").text for elem in all_articles]
    return all_page_ids, all_page_urls, all_page_titles, all_page_author_date
def clean_urls_root(urls, root="okdiario.com"):
    urls = pd.Series(urls)
    urls.dropna(inplace=True)
    urls_root = urls[urls.str.contains(root)]
    return urls_root

In [76]:
all_page_urls_elements = driver.find_elements(By.XPATH, "//a[@href]")
all_page_urls = [elem.get_attribute("href") for elem in all_page_urls_elements]
len(all_page_urls), all_page_urls[:5]

(396,
 ['https://okdiario.com/',
  'https://okdiario.com/ultimas-noticias/',
  'https://okdiario.com/espana/',
  'https://okdiario.com/madrid/',
  'https://okdiario.com/cataluna/'])

Analyze urls

In [154]:
urls = pd.Series(all_page_urls)
urls.dropna(inplace=True)
urls.head()

0                     https://okdiario.com/
1    https://okdiario.com/ultimas-noticias/
2              https://okdiario.com/espana/
3              https://okdiario.com/madrid/
4            https://okdiario.com/cataluna/
dtype: object

In [162]:
# Extract urls with base root
is_base_root = urls.str.contains("okdiario.com")
print(is_base_root.sum(), "roots")
urls_root = urls[is_base_root]
urls_root.head(12)

0                          https://okdiario.com/
1         https://okdiario.com/ultimas-noticias/
2                   https://okdiario.com/espana/
3                   https://okdiario.com/madrid/
4                 https://okdiario.com/cataluna/
5                https://okdiario.com/andalucia/
6               https://okdiario.com/pais-vasco/
7     https://okdiario.com/comunidad-valenciana/
8                  https://navarra.okdiario.com/
9                 https://okdiario.com/baleares/
10                https://okdiario.com/economia/
11        https://okdiario.com/economia/consumo/
dtype: object

In [157]:
# Extract non-root levels
urls_nonroot = urls_root.str.extract(("//okdiario.com/(.*)"), expand=False)
urls_nonroot.str.split("/").head(20)

0                                         []
1                       [ultimas-noticias, ]
2                                 [espana, ]
3                                 [madrid, ]
4                               [cataluna, ]
5                              [andalucia, ]
6                             [pais-vasco, ]
7                   [comunidad-valenciana, ]
8                                        NaN
9                               [baleares, ]
10                              [economia, ]
11                     [economia, consumo, ]
12                      [economia, empleo, ]
13                         [investigacion, ]
14                      [diariomadridista, ]
15    [diariomadridista, ultimas-noticias, ]
16         [diariomadridista, real-madrid, ]
17                              [deportes, ]
18                      [deportes, futbol, ]
19             [deportes, atletico-madrid, ]
dtype: object

In [161]:
# Frequencies of non-root levels in urls
nonroot_levels_freq = urls.str.split("/").str.len()
print(nonroot_levels_freq.value_counts())
levels_gt_2 = urls[nonroot_levels_freq > 2]
levels_gt_2

5    287
6     92
7      9
4      7
8      1
dtype: int64


0                       https://okdiario.com/
1      https://okdiario.com/ultimas-noticias/
2                https://okdiario.com/espana/
3                https://okdiario.com/madrid/
4              https://okdiario.com/cataluna/
                        ...                  
391    https://okdiario.com/politica-cookies/
392            https://okdiario.com/archivos/
393            https://okdiario.com/noticias/
394                 https://okdiario.com/feed
395       https://okdiario.com/quienes-somos/
Length: 396, dtype: object

#### In-depth case

In [169]:
deep_1_url = "https://okdiario.com/espana/"

In [170]:
DEFAULT_WAIT_TIME = 5
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=None)
driver.get(deep_1_url)
clickable_condition = EC.element_to_be_clickable((By.XPATH, "//button/span[contains(text(), 'Aceptar ')]"))
WebDriverWait(driver, DEFAULT_WAIT_TIME).until(clickable_condition).click()

In [176]:
deep_1_urls = extract_articles_meta()
len(deep_1_urls)

MaxRetryError: HTTPConnectionPool(host='localhost', port=45637): Max retries exceeded with url: /session/7e100674bb96c649192dd348ec1e1b8c/elements (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000025E57963700>: Failed to establish a new connection: [WinError 10061] No se puede establecer una conexión ya que el equipo de destino denegó expresamente dicha conexión'))

### Xataka

In [251]:
reg_date = r"[0-9]{1,2} de [a-zA-Z]{4,9} de 20[0-9]{2}$"
test_string = "71 de mayo de 2023"
re.match(reg_date, test_string), test_string.split(" de ")

(<re.Match object; span=(0, 18), match='71 de mayo de 2023'>,
 ['71', 'mayo', '2023'])

In [234]:
month_sp_to_en = {
    'enero': 'january',
    'febrero': 'february',
    'marzo': 'march',
    'abril': 'april',
    'mayo': 'may',
    'junio': 'june',
    'julio': 'july',
    'agosto': 'august',
    'septiembre': 'september',
    'octubre': 'october',
    'noviembre': 'november',
    'diciembre': 'december'
}

In [368]:
def xataka_extract_articles_meta():
    archive_elem = driver.find_element(By.XPATH, "//div[@class='list-archive']")
    dates_and_titles = archive_elem.text
    reg_str = r"[0-9]{1,2} de [a-zA-Z]{4,9} de 20[0-9]{2}$"
    pattern = re.compile(reg_str)
    titles = []
    articles_meta = {}
    old_date = None
    for data in dates_and_titles.split("\n"):
        result = pattern.match(data)
        if result:
            #print(pattern.match(data))
            day, month, year = data.split(" de ")
            month = month_sp_to_en[month]
            #print(old_date)
            if old_date is not None:
                print(old_date, len(titles))
                articles_meta[old_date] = {"titles": titles}
                titles = []
            old_date = "-".join([day, month, year])
        else:
            titles.append(data)
    # Last set of titles is not dict-set in the loop
    articles_meta[old_date] = {"titles": titles}
    
    ul_li_a_elems = archive_elem.find_elements(By.XPATH, "ul/li/a")
    hrefs = [elem.get_attribute("href") for elem in ul_li_a_elems]
    for (k, _), url in zip(articles_meta.items(), hrefs):
        articles_meta[k].update({"url": url})
    return articles_meta
def xataka_clean_urls_root(urls, root="xataka.com"):
    urls = pd.Series(urls)
    urls.dropna(inplace=True)
    urls_root = urls[urls.str.contains(root)]
    return urls_root

In [367]:
archive_elem = driver.find_element(By.XPATH, "//div[@class='list-archive']")
x = archive_elem.text
reg_str = r"[0-9]{1,2} de [a-zA-Z]{4,9} de 20[0-9]{2}$"
pattern = re.compile(reg_str)
titles = []
articles_meta = {}
old_date = None
for data in x.split("\n"):
    result = pattern.match(data)
    if result:
        #print(pattern.match(data))
        day, month, year = data.split(" de ")
        month = month_sp_to_en[month]
        #print(old_date)
        if old_date is not None:
            print(old_date, len(titles))
            articles_meta[old_date] = {"titles": titles}
            titles = []
        old_date = "-".join([day, month, year])
    else:
        titles.append(data)
# Last set of titles is not dict-set in the loop
articles_meta[old_date] = {"titles": titles}
ul_li_a_elems = archive_elem.find_elements(By.XPATH, "ul/li/a")
hrefs = [elem.get_attribute("href") for elem in ul_li_a_elems]
for (k, _), url in zip(articles_meta.items(), hrefs):
    articles_meta[k].update({"url": url})
articles_meta

16-may-2023 30
15-may-2023 43
14-may-2023 15
13-may-2023 17
12-may-2023 30
11-may-2023 43
10-may-2023 53
9-may-2023 57
8-may-2023 42
7-may-2023 19
6-may-2023 17
5-may-2023 46
4-may-2023 55
3-may-2023 50
2-may-2023 44


{'16-may-2023': {'titles': ['No diga "embalse", diga "enorme batería hidroeléctrica": España tiene que repensar sus pantanos antes de que sea tarde',
   'Xiaomi tiene un secador, bueno, bonito y barato que hidrata el pelo a un precio nunca visto',
   'Este portátil convertible Dell con i7 y 16 GB de RAM tiene descuentazo de 350 euros que lo deja a precio mínimo',
   'Las primeras víctimas de las sanciones de EEUU a China: los chips de 14 y 7 nm de SMIC',
   'PowerA MOGA XP-Ultra: así es el primer mando cuatro en uno de terceros que funciona sin cables con las Xbox Series X|S',
   'Ya podemos viajar a bordo de un taxi autónomo de Waymo gracias a este vídeo en el interior de uno de ellos',
   'Con este ratón gaming Corsair casi a mitad de precio mejorarás tu precisión: tiene seis botones programables y gran sensibilidad',
   'Apple lanza \'Personal Voice\', una opción que permitirá que el iPhone "hable" con tu propia voz',
   'La freidora de aire pequeña de Cosori vuelve a bajar de preci

In [347]:
year = "2023"
month="5"
xataka_url_archives_1 = f"https://www.xataka.com/archivos/{year}/{month}"
xataka_url_archives_1

'https://www.xataka.com/archivos/2023/5'

In [348]:
DEFAULT_WAIT_TIME = 5
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=None)
driver.get(xataka_url_archives_1)
clickable_condition = EC.element_to_be_clickable((By.XPATH, "//button/span[contains(text(), 'Aceptar ')]"))
WebDriverWait(driver, DEFAULT_WAIT_TIME).until(clickable_condition).click()

In [369]:
news_metadata = xataka_extract_articles_meta()

16-may-2023 30
15-may-2023 43
14-may-2023 15
13-may-2023 17
12-may-2023 30
11-may-2023 43
10-may-2023 53
9-may-2023 57
8-may-2023 42
7-may-2023 19
6-may-2023 17
5-may-2023 46
4-may-2023 55
3-may-2023 50
2-may-2023 44


In [383]:
(news_metadata["16-may-2023"]["url"])

'https://www.xataka.com/ecologia-y-naturaleza/no-digas-embalse-di-enorme-bateria-hidroelectrica-espana-tiene-que-repensar-sus-pantanos-antes-que-sea-tarde'

In [385]:
driver.get((news_metadata["16-may-2023"]["url"]))

In [376]:
list(map(lambda x,y: len(x["titles"]), news_metadata))

TypeError: <lambda>() missing 1 required positional argument: 'y'

### Otros

In [ ]:
#company_name = "Goiko grill"
journal_name = "20minutos"

DEFAULT_WAIT_TIME = 5
# Open browser
options = webdriver.ChromeOptions() 
options.headless = True

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=None)
driver.implicitly_wait(DEFAULT_WAIT_TIME)

# Go to Reviews of the company
driver.get("https://www.google.es/maps")
#driver.fullscreen_window()

accept_all = driver.find_element(By.XPATH, "//button[@aria-label='Aceptar todo']")
accept_all.send_keys(Keys.ENTER)
"""
c = 0
search_box_element = driver.find_element(By.ID, "searchboxinput")
while c < len(company_name):
    search_box_element.send_keys(Keys.BACKSPACE)
    c += 1
search_box_element.send_keys(company_name)"""

area_menu = driver.find_element(By.XPATH, "//ul[@class='default-menu expandable']/li")
search_accept_element.click()

time.sleep(5)
search_results_element = driver.find_element(By.XPATH, "//div[@role='feed']")
search_results_element

time.sleep(2)
article_elements = search_results_element.find_elements(By.XPATH, "//div[@role='article']")
first_article_element = article_elements[0]
first_article_element.click()

time.sleep(2)
clickable_condition = EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'reseñas')]"))
reviews_element = WebDriverWait(driver, DEFAULT_WAIT_TIME).until(clickable_condition)
reviews_element.click()

#time.sleep(2)

#driver.find_element(By.XPATH, "//button[contains(text(), 'reseñas')]").click()
#reviews_button_element = driver.find_element(By.XPATH, "//div[@role='button']")
#reviews_button_element.click()

total_reviews_text = driver.find_element(By.CLASS_NAME, "PPCwl").text
n_total_reviews = total_reviews_text.split("\n")[-1][:-8].replace(".", "")
n_total_reviews = int(n_total_reviews)

# About 10 reviews are load per load order
n_reviews_per_load = 10
n_estimated_loads = math.ceil(n_total_reviews / 10)
n_estimated_loads

n_total_reviews